#Mount Google Drive

In [2]:
# from google.colab import drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


#Set up dependencies

In [ ]:
!pip install einops
!pip install --upgrade --no-cache-dir gdown
!pip install --upgrade --no-cache-dir xlrd
!pip install av --upgrade --no-cache-dir xlrd
!pip install torchvision
!pip install torchvideo
!pip install timm

#Import the necessary packages

In [ ]:
import sys
import os

#get the absolute path of the current directory
current_dir = os.path.abspath(os.path.dirname(__file__))

#add the paths of the folders containing 'data' and 'config_' to sys.path
sys.path.append(os.path.join(current_dir, 'PFE-HAR-master'))
sys.path.append(os.path.join(current_dir, 'PFE-HAR-master', 'data'))
sys.path.append(os.path.join(current_dir, 'PFE-HAR-master', 'config_'))
sys.path.append(os.path.join(current_dir, 'train_dataset'))


In [ ]:
import torch
import os
from torch import nn, einsum
from einops import rearrange
from einops.layers.torch import Rearrange
from einops import repeat
from einops.layers.torch import Reduce
import numpy as np
import pandas as pd
import torchvision
import subprocess
import time
import sys
import importlib
import time

import config_ as cfg

from torch.utils.data import DataLoader
from torch.utils.data import Dataset
from torchvision import transforms


cfg.DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("Device being used:", cfg.DEVICE)

torch.autograd.set_detect_anomaly(False)
torch.autograd.profiler.profile(False)
torch.autograd.profiler.emit_nvtx(False)
torch.backends.cudnn.benchmark = True

#Check Directory Functions

In [ ]:
def count_files_in_directory(directory):
    try:
        files = os.listdir(directory)
        num_files = len(files)
        return num_files
    except FileNotFoundError:
        print(f"Error: Directory '{directory}' not found.")
        return 0

DIRECTORY_PATH = ''

num_files = count_files_in_directory(DIRECTORY_PATH)
print(f"Total number of files in directory '{DIRECTORY_PATH}': {num_files}")

#Config Section

In [ ]:
cfg.IMG_SIZE = 224
cfg.BATCH_SIZE = 4
cfg.DIM = 512
cfg.FPS = 30
cfg.DATA_PATH = "./train_dataset"
cfg.NUM_WORKERS = 0
cfg.PIN_MEMORY = False
cfg.IS_TRAIN = True
cfg.NB_EPOCHES = 100

cfg.BLOCKS_DIM = [64, 128, 512, 512]
#cfg.DEVICE = "cuda:0"
cfg.DROP_OUT = 0.5
cfg.DROP_PATH = 0.5
cfg.MLP_RATIO = 4.
cfg.NUM_CLASSES = 41

cfg.LEARNING_RATE = 0.001
cfg.WEIGHT_DECAY = 0.0005

cfg.MODEL_PATH = ""

cfg.DATASET_NAME = ''

if cfg.DATASET_NAME == 'NTU_RGB_D_60':
    cfg.DATA_DIR = os.path.join(cfg.DATA_PATH, 'NTU_RGB_D_60/RGB_videos/nturgb+d_rgb')
    cfg.LABELS_DIR = os.path.join(cfg.DATA_PATH, 'NTU_RGB_D_60/Labels/labels')
    cfg.ACTIONS_File = os.path.join(cfg.DATA_PATH, 'NTU_RGB_D_60/Actions.xlsx')
elif cfg.DATASET_NAME == 'PKU_MMD_1':
    cfg.DATA_DIR = os.path.join(cfg.DATA_PATH, 'PKU_MMD_1/Data/RGB_videos')
    cfg.LABELS_DIR = os.path.join(cfg.DATA_PATH, 'PKU_MMD_1/Data/Labels/labels')
    cfg.ACTIONS_File = os.path.join(cfg.DATA_PATH, 'PKU_MMD_1/Data/Labels/labels/Actions.xlsx')
elif cfg.DATASET_NAME == 'PKU_MMD_2':
    cfg.DATA_DIR = os.path.join(cfg.DATA_PATH, 'PKU_MMD_2/Data/RGB_videos/RGB_VIDEO_v2')
    cfg.LABELS_DIR = os.path.join(cfg.DATA_PATH, 'PKU_MMD_2/Data/Labels/labels')
    cfg.ACTIONS_File = os.path.join(cfg.DATA_PATH, 'PKU_MMD_2/Data/Labels/labels/Actions.xlsx')
elif cfg.DATASET_NAME == 'HMDB51':
    DATA_DIR = os.path.join(cfg.DATA_PATH, 'HMDB51/videos')
    LABELS_FILE = os.path.join(cfg.DATA_PATH, 'HMDB51/labels')
    ACTIONS_File = None  # You can configure an actions file if needed

elif cfg.DATASET_NAME == 'UCF101':
    DATA_DIR = os.path.join(cfg.DATA_PATH, 'UCF101/videos')
    LABELS_FILE = os.path.join(cfg.DATA_PATH, 'UCF101/labels')
    ACTIONS_File = None  # You can configure an actions file if needed

elif cfg.DATASET_NAME == 'ETRI3D':
    DATA_DIR = os.path.join(cfg.DATA_PATH, 'ETRI3D/videos')
    LABELS_FILE = None  # Ensure your DataLoader handles this configuration
    ACTIONS_File = None  # You can configure an actions file if needed
else:
    raise ValueError(f"Unknown dataset: {cfg.DATASET_NAME}")

#Empty Cash Call

In [ ]:
torch.cuda.empty_cache()

#Load Data

In [ ]:
import data.new_datasets
importlib.reload(data.new_datasets)
from data.new_datasets import PKUMMDPhase1Dataset, PKUMMDPhase2Dataset, NTURGBD60Dataset, HMDB51Dataset, UCF101Dataset, ETRI3DDataset
from torch.utils.data import DataLoader, random_split
from data_loader import get_data_loader, collate_fn
# from data.data_loader import collate_fn
import time
from tqdm import tqdm

In [ ]:
# Charger les données
subset_size = 500
try:
    all_data = get_data_loader(cfg.DATASET_NAME, cfg.DATA_DIR, cfg.LABELS_DIR, cfg.ACTIONS_File, batch_size=cfg.BATCH_SIZE, transform=transform, mode='all', subset_size=subset_size, collate_fn=collate_fn, img_size=cfg.IMG_SIZE)
except Exception as e:
    print(f"Error loading data: {e}")
    raise

if len(all_data.dataset) == 0:
    raise ValueError("No data loaded. Check file paths and dataset configurations.")

##Divide data into train and test data

In [ ]:
#define the sizes for training and testing datasets
train_size = int(0.7 * len(all_data.dataset))  # 70% pour l'entraînement
test_size = len(all_data.dataset) - train_size  # 30% pour le test

#split the data into training and testing datasets
train_dataset, test_dataset = random_split(all_data.dataset, [train_size, test_size])

#create DataLoaders for training and testing
train_loader = DataLoader(train_dataset, batch_size=cfg.BATCH_SIZE, shuffle=True, collate_fn=collate_fn)
test_loader = DataLoader(test_dataset, batch_size=cfg.BATCH_SIZE, shuffle=False, collate_fn=collate_fn)

print(f"Nombre total de vidéos pour l'entraînement : {len(train_dataset)}")
print(f"Nombre total de vidéos pour le test : {len(test_dataset)}")

#Construct the Model

In [ ]:
from model import Model

# Construct the model
if cfg.MODEL_PATH == "":
    print("============== Construct The Model ========")
    model = Model(cfg.NUM_CLASSES).to(cfg.DEVICE)
else:
    print("============== Load The Model ========")
    model = torch.load(cfg.MODEL_PATH).to(cfg.DEVICE)

torch.cuda.empty_cache()

#Train - Test

In [ ]:
from train_evaluate import TrainAndEvaluate

# Train
if cfg.MODEL_PATH == "":
    print("=========== Train ======== \n")
    loss_list, training_accuracy_list = trainAndEvaluate.train(train_loader, cfg.LEARNING_RATE, cfg.WEIGHT_DECAY,
                                                               intermediate_result_step=10, print_epoch_result_step=1)
    # Save the trained model
    trainAndEvaluate.save_model('./saved_model.pth')
    cfg.MODEL_PATH = './saved_model.pth'
else:
    print("=========== Evaluate ======== \n")
    # Evaluate the model on the test data
    trainAndEvaluate.test(test_loader, intermediate_result_step=10)


#Visualization

##Import and Initialize the Visualization Class

In [ ]:
from visualization import Visualization

# Initialiser l'objet de visualisation avec le modèle
visualization = Visualization(model)


In [ ]:
#plot the training loss curve
visualization.plot_loss(loss_list)


In [ ]:
#select a video sample from the DataLoader
video_sample = next(iter(train_loader))[0][0]  # Get the first video sample from the training DataLoader


In [ ]:
#visualize the raw video (before processing)
video_animation = visualization.plot_video(video_sample)
video_animation


In [ ]:
#set hooks to capture intermediate activations
visualization.set_hooks()

#perform a forward pass through the model to capture activations
visualization.perform_forward(video_sample)


In [ ]:
#visualize the output of the first CNN layer
visualization.plot_first_cnn_layer()


In [ ]:
#visualize the output of the spatial CNN block
spatial_bloc_animation = visualization.plot_spatial_bloc_output()
spatial_bloc_animation


In [ ]:
#visualize the attention effect as a heatmap overlaid on the original video
attention_animation = visualization.plot_attention(video_sample)
attention_animation
